# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [8]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text,target_id_text = [],[]
    for line_id,lines in enumerate(source_text.split('\n')):
        source_id_text.append([])
        for word in lines.split():
            source_id_text[line_id].append(source_vocab_to_int[word])
    for line_id,lines in enumerate(target_text.split('\n')):
        target_id_text.append([])
        for word in lines.split():
            target_id_text[line_id].append(target_vocab_to_int[word])
        target_id_text[line_id].append(target_vocab_to_int['<EOS>'])
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [3]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None,None], name= "input")
    targets = tf.placeholder(tf.int32, shape=[None,None], name= "targets")
    learning_rate = tf.placeholder(tf.float32, name= "learning_rate")
    keep_prob = tf.placeholder(tf.float32, name= "keep_prob")
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_seq_len = tf.reduce_max(target_seq_lenth, name= 'max_target_seq_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    return (inputs, targets, learning_rate, keep_prob, target_seq_lenth, max_target_seq_len, source_seq_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "C:\\Users\\admin\\Anaconda3\\envs\\tensorflow-gpu\\lib\\runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "C:\\Users\\admin\\Anaconda3\\envs\\tensorflow-gpu\\lib\\runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "C:\\Users\\admin\\Anaconda3\\envs\\tensorflow-gpu\\lib\\site-packages\\ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "C:\\Users\\admin\\Anaconda3\\envs\\tensorflow-gpu\\lib\\site-packages\\traitlets\\config\\application.py", line 658, in launch_instance\n    app.start()', 'File "C:\\Users\\admin\\Anaconda3\\envs\\tensorflow-gpu\\lib\\site-packages\\ipykernel\\kernelapp.py", line 477, in start\n    ioloo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [4]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    processed_target_data = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [5]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    multi_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    output, state = tf.nn.dynamic_rnn(multi_lstm, embed, source_sequence_length, dtype=tf.float32)
    return (output, state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainig_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, trainig_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [23]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, embedding_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_target_sequence_length)
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [28]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    dense_layer = Dense(target_vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        scope.reuse_variables()
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [9]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,
                                   source_sequence_length, source_vocab_size, enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [10]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/1077 - Train Accuracy: 0.2215, Validation Accuracy: 0.3050, Loss: 5.6340
Epoch   0 Batch    2/1077 - Train Accuracy: 0.2064, Validation Accuracy: 0.3050, Loss: 5.4603
Epoch   0 Batch    3/1077 - Train Accuracy: 0.2387, Validation Accuracy: 0.3050, Loss: 5.2039
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2246, Validation Accuracy: 0.3054, Loss: 5.0285
Epoch   0 Batch    5/1077 - Train Accuracy: 0.2605, Validation Accuracy: 0.3054, Loss: 4.7389
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2430, Validation Accuracy: 0.3054, Loss: 4.6334
Epoch   0 Batch    7/1077 - Train Accuracy: 0.2258, Validation Accuracy: 0.3054, Loss: 4.6077
Epoch   0 Batch    8/1077 - Train Accuracy: 0.2320, Validation Accuracy: 0.3061, Loss: 4.5026
Epoch   0 Batch    9/1077 - Train Accuracy: 0.2379, Validation Accuracy: 0.3125, Loss: 4.4111
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2146, Validation Accuracy: 0.3260, Loss: 4.4901
Epoch   0 Batch   11/1077 - Train Accuracy: 0.2898, Validati

Epoch   0 Batch   90/1077 - Train Accuracy: 0.4090, Validation Accuracy: 0.4638, Loss: 2.7411
Epoch   0 Batch   91/1077 - Train Accuracy: 0.4673, Validation Accuracy: 0.4663, Loss: 2.4369
Epoch   0 Batch   92/1077 - Train Accuracy: 0.4345, Validation Accuracy: 0.4652, Loss: 2.6059
Epoch   0 Batch   93/1077 - Train Accuracy: 0.4039, Validation Accuracy: 0.4641, Loss: 2.6465
Epoch   0 Batch   94/1077 - Train Accuracy: 0.4336, Validation Accuracy: 0.4688, Loss: 2.5914
Epoch   0 Batch   95/1077 - Train Accuracy: 0.4479, Validation Accuracy: 0.4698, Loss: 2.4990
Epoch   0 Batch   96/1077 - Train Accuracy: 0.4160, Validation Accuracy: 0.4712, Loss: 2.5750
Epoch   0 Batch   97/1077 - Train Accuracy: 0.4371, Validation Accuracy: 0.4744, Loss: 2.5698
Epoch   0 Batch   98/1077 - Train Accuracy: 0.4583, Validation Accuracy: 0.4727, Loss: 2.4605
Epoch   0 Batch   99/1077 - Train Accuracy: 0.4078, Validation Accuracy: 0.4815, Loss: 2.6478
Epoch   0 Batch  100/1077 - Train Accuracy: 0.4258, Validati

Epoch   0 Batch  179/1077 - Train Accuracy: 0.4515, Validation Accuracy: 0.5067, Loss: 2.1491
Epoch   0 Batch  180/1077 - Train Accuracy: 0.4492, Validation Accuracy: 0.4915, Loss: 2.0318
Epoch   0 Batch  181/1077 - Train Accuracy: 0.4250, Validation Accuracy: 0.4719, Loss: 2.0515
Epoch   0 Batch  182/1077 - Train Accuracy: 0.4453, Validation Accuracy: 0.4837, Loss: 2.0337
Epoch   0 Batch  183/1077 - Train Accuracy: 0.4293, Validation Accuracy: 0.5000, Loss: 2.1311
Epoch   0 Batch  184/1077 - Train Accuracy: 0.4742, Validation Accuracy: 0.5000, Loss: 2.0305
Epoch   0 Batch  185/1077 - Train Accuracy: 0.4121, Validation Accuracy: 0.4780, Loss: 2.0170
Epoch   0 Batch  186/1077 - Train Accuracy: 0.4371, Validation Accuracy: 0.4851, Loss: 2.1037
Epoch   0 Batch  187/1077 - Train Accuracy: 0.4395, Validation Accuracy: 0.4943, Loss: 2.0594
Epoch   0 Batch  188/1077 - Train Accuracy: 0.4426, Validation Accuracy: 0.4886, Loss: 2.0123
Epoch   0 Batch  189/1077 - Train Accuracy: 0.4207, Validati

Epoch   0 Batch  267/1077 - Train Accuracy: 0.4847, Validation Accuracy: 0.5018, Loss: 1.5044
Epoch   0 Batch  268/1077 - Train Accuracy: 0.4820, Validation Accuracy: 0.5043, Loss: 1.5975
Epoch   0 Batch  269/1077 - Train Accuracy: 0.4433, Validation Accuracy: 0.5011, Loss: 1.7112
Epoch   0 Batch  270/1077 - Train Accuracy: 0.4531, Validation Accuracy: 0.5117, Loss: 1.6873
Epoch   0 Batch  271/1077 - Train Accuracy: 0.4793, Validation Accuracy: 0.5117, Loss: 1.6166
Epoch   0 Batch  272/1077 - Train Accuracy: 0.4885, Validation Accuracy: 0.5096, Loss: 1.5507
Epoch   0 Batch  273/1077 - Train Accuracy: 0.5004, Validation Accuracy: 0.5259, Loss: 1.5152
Epoch   0 Batch  274/1077 - Train Accuracy: 0.4896, Validation Accuracy: 0.5234, Loss: 1.5608
Epoch   0 Batch  275/1077 - Train Accuracy: 0.4914, Validation Accuracy: 0.5210, Loss: 1.5007
Epoch   0 Batch  276/1077 - Train Accuracy: 0.4594, Validation Accuracy: 0.5241, Loss: 1.5973
Epoch   0 Batch  277/1077 - Train Accuracy: 0.4955, Validati

Epoch   0 Batch  356/1077 - Train Accuracy: 0.4746, Validation Accuracy: 0.5092, Loss: 1.3519
Epoch   0 Batch  357/1077 - Train Accuracy: 0.5301, Validation Accuracy: 0.5398, Loss: 1.2345
Epoch   0 Batch  358/1077 - Train Accuracy: 0.4840, Validation Accuracy: 0.5529, Loss: 1.3662
Epoch   0 Batch  359/1077 - Train Accuracy: 0.5113, Validation Accuracy: 0.5561, Loss: 1.2920
Epoch   0 Batch  360/1077 - Train Accuracy: 0.5172, Validation Accuracy: 0.5533, Loss: 1.3070
Epoch   0 Batch  361/1077 - Train Accuracy: 0.5148, Validation Accuracy: 0.5415, Loss: 1.3156
Epoch   0 Batch  362/1077 - Train Accuracy: 0.5074, Validation Accuracy: 0.5344, Loss: 1.2977
Epoch   0 Batch  363/1077 - Train Accuracy: 0.4512, Validation Accuracy: 0.5352, Loss: 1.3439
Epoch   0 Batch  364/1077 - Train Accuracy: 0.4918, Validation Accuracy: 0.5305, Loss: 1.3257
Epoch   0 Batch  365/1077 - Train Accuracy: 0.4633, Validation Accuracy: 0.5138, Loss: 1.3300
Epoch   0 Batch  366/1077 - Train Accuracy: 0.4816, Validati

...

Epoch   9 Batch  655/1077 - Train Accuracy: 0.9371, Validation Accuracy: 0.9453, Loss: 0.0624
Epoch   9 Batch  656/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9553, Loss: 0.0550
Epoch   9 Batch  657/1077 - Train Accuracy: 0.9696, Validation Accuracy: 0.9503, Loss: 0.0469
Epoch   9 Batch  658/1077 - Train Accuracy: 0.9777, Validation Accuracy: 0.9489, Loss: 0.0311
Epoch   9 Batch  659/1077 - Train Accuracy: 0.9635, Validation Accuracy: 0.9489, Loss: 0.0554
Epoch   9 Batch  660/1077 - Train Accuracy: 0.9742, Validation Accuracy: 0.9485, Loss: 0.0469
Epoch   9 Batch  661/1077 - Train Accuracy: 0.9740, Validation Accuracy: 0.9538, Loss: 0.0389
Epoch   9 Batch  662/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9538, Loss: 0.0471
Epoch   9 Batch  663/1077 - Train Accuracy: 0.9586, Validation Accuracy: 0.9531, Loss: 0.0441
Epoch   9 Batch  664/1077 - Train Accuracy: 0.9535, Validation Accuracy: 0.9531, Loss: 0.0554
Epoch   9 Batch  665/1077 - Train Accuracy: 0.9543, Validati

Epoch   9 Batch  744/1077 - Train Accuracy: 0.9576, Validation Accuracy: 0.9297, Loss: 0.0515
Epoch   9 Batch  745/1077 - Train Accuracy: 0.9645, Validation Accuracy: 0.9290, Loss: 0.0486
Epoch   9 Batch  746/1077 - Train Accuracy: 0.9652, Validation Accuracy: 0.9339, Loss: 0.0451
Epoch   9 Batch  747/1077 - Train Accuracy: 0.9719, Validation Accuracy: 0.9343, Loss: 0.0429
Epoch   9 Batch  748/1077 - Train Accuracy: 0.9414, Validation Accuracy: 0.9318, Loss: 0.0524
Epoch   9 Batch  749/1077 - Train Accuracy: 0.9895, Validation Accuracy: 0.9393, Loss: 0.0483
Epoch   9 Batch  750/1077 - Train Accuracy: 0.9426, Validation Accuracy: 0.9435, Loss: 0.0433
Epoch   9 Batch  751/1077 - Train Accuracy: 0.9703, Validation Accuracy: 0.9482, Loss: 0.0585
Epoch   9 Batch  752/1077 - Train Accuracy: 0.9457, Validation Accuracy: 0.9528, Loss: 0.0494
Epoch   9 Batch  753/1077 - Train Accuracy: 0.9621, Validation Accuracy: 0.9460, Loss: 0.0605
Epoch   9 Batch  754/1077 - Train Accuracy: 0.9445, Validati

Epoch   9 Batch  832/1077 - Train Accuracy: 0.9621, Validation Accuracy: 0.9467, Loss: 0.0471
Epoch   9 Batch  833/1077 - Train Accuracy: 0.9598, Validation Accuracy: 0.9403, Loss: 0.0553
Epoch   9 Batch  834/1077 - Train Accuracy: 0.9773, Validation Accuracy: 0.9354, Loss: 0.0530
Epoch   9 Batch  835/1077 - Train Accuracy: 0.9676, Validation Accuracy: 0.9389, Loss: 0.0527
Epoch   9 Batch  836/1077 - Train Accuracy: 0.9749, Validation Accuracy: 0.9379, Loss: 0.0435
Epoch   9 Batch  837/1077 - Train Accuracy: 0.9480, Validation Accuracy: 0.9432, Loss: 0.0647
Epoch   9 Batch  838/1077 - Train Accuracy: 0.9508, Validation Accuracy: 0.9382, Loss: 0.0479
Epoch   9 Batch  839/1077 - Train Accuracy: 0.9789, Validation Accuracy: 0.9396, Loss: 0.0418
Epoch   9 Batch  840/1077 - Train Accuracy: 0.9781, Validation Accuracy: 0.9293, Loss: 0.0378
Epoch   9 Batch  841/1077 - Train Accuracy: 0.9559, Validation Accuracy: 0.9293, Loss: 0.0645
Epoch   9 Batch  842/1077 - Train Accuracy: 0.9715, Validati

Epoch   9 Batch  920/1077 - Train Accuracy: 0.9574, Validation Accuracy: 0.9407, Loss: 0.0459
Epoch   9 Batch  921/1077 - Train Accuracy: 0.9500, Validation Accuracy: 0.9421, Loss: 0.0546
Epoch   9 Batch  922/1077 - Train Accuracy: 0.9509, Validation Accuracy: 0.9474, Loss: 0.0560
Epoch   9 Batch  923/1077 - Train Accuracy: 0.9753, Validation Accuracy: 0.9478, Loss: 0.0400
Epoch   9 Batch  924/1077 - Train Accuracy: 0.9704, Validation Accuracy: 0.9382, Loss: 0.0582
Epoch   9 Batch  925/1077 - Train Accuracy: 0.9699, Validation Accuracy: 0.9311, Loss: 0.0446
Epoch   9 Batch  926/1077 - Train Accuracy: 0.9426, Validation Accuracy: 0.9311, Loss: 0.0520
Epoch   9 Batch  927/1077 - Train Accuracy: 0.9418, Validation Accuracy: 0.9414, Loss: 0.0575
Epoch   9 Batch  928/1077 - Train Accuracy: 0.9367, Validation Accuracy: 0.9407, Loss: 0.0450
Epoch   9 Batch  929/1077 - Train Accuracy: 0.9598, Validation Accuracy: 0.9407, Loss: 0.0486
Epoch   9 Batch  930/1077 - Train Accuracy: 0.9531, Validati

Epoch   9 Batch 1009/1077 - Train Accuracy: 0.9777, Validation Accuracy: 0.9403, Loss: 0.0332
Epoch   9 Batch 1010/1077 - Train Accuracy: 0.9648, Validation Accuracy: 0.9471, Loss: 0.0413
Epoch   9 Batch 1011/1077 - Train Accuracy: 0.9684, Validation Accuracy: 0.9492, Loss: 0.0382
Epoch   9 Batch 1012/1077 - Train Accuracy: 0.9736, Validation Accuracy: 0.9379, Loss: 0.0396
Epoch   9 Batch 1013/1077 - Train Accuracy: 0.9717, Validation Accuracy: 0.9379, Loss: 0.0388
Epoch   9 Batch 1014/1077 - Train Accuracy: 0.9543, Validation Accuracy: 0.9325, Loss: 0.0423
Epoch   9 Batch 1015/1077 - Train Accuracy: 0.9352, Validation Accuracy: 0.9379, Loss: 0.0589
Epoch   9 Batch 1016/1077 - Train Accuracy: 0.9602, Validation Accuracy: 0.9379, Loss: 0.0471
Epoch   9 Batch 1017/1077 - Train Accuracy: 0.9618, Validation Accuracy: 0.9386, Loss: 0.0502
Epoch   9 Batch 1018/1077 - Train Accuracy: 0.9695, Validation Accuracy: 0.9474, Loss: 0.0409
Epoch   9 Batch 1019/1077 - Train Accuracy: 0.9424, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence_id = []
    for word in sentence.lower().split():
        if word in vocab_to_int.keys():
            sentence_id.append(vocab_to_int[word])
        else:
            sentence_id.append(vocab_to_int['<UNK>'])
    return sentence_id


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [21]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [96, 168, 123, 74, 25, 115, 141]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [159, 228, 312, 152, 217, 235, 167, 315, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.